In [1]:
import numpy as np
import tensorflow as tf
from collections import Counter
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("data/fashion")
X, y, k = mnist.test.images, mnist.test.labels, 10
# 选择随机点作为起始位置
start_pos = tf.Variable(X[np.random.randint(X.shape[0], size=k), :], dtype=tf.float32)
centroids = tf.Variable(start_pos.initialized_value(), 'S', dtype=tf.float32)

# 填充点
points = tf.Variable(X, 'X', dtype=tf.float32)
ones_like = tf.ones((points.get_shape()[0], 1))
prev_assignments = tf.Variable(tf.zeros((points.get_shape()[0], ), dtype=tf.int64))

# 找到所有点之间的距离
p1 = tf.matmul(
    tf.expand_dims(tf.reduce_sum(tf.square(points), 1), 1),
    tf.ones(shape=(1, k))
)
p2 = tf.transpose(tf.matmul(
    tf.reshape(tf.reduce_sum(tf.square(centroids), 1), shape=[-1, 1]),
    ones_like,
    transpose_b=True
))
distance = tf.sqrt(tf.add(p1, p2) - 2 * tf.matmul(points, centroids, transpose_b=True))

# 将每个点分配到最近的质心
point_to_centroid_assignment = tf.argmin(distance, axis=1)

# 重新计算中心
total = tf.unsorted_segment_sum(points, point_to_centroid_assignment, k)
count = tf.unsorted_segment_sum(ones_like, point_to_centroid_assignment, k)
means = total / count

# 如果当前和以前的作业有任何差异，请继续
is_continue = tf.reduce_any(tf.not_equal(point_to_centroid_assignment, prev_assignments))

with tf.control_dependencies([is_continue]):
    loop = tf.group(centroids.assign(means), prev_assignments.assign(point_to_centroid_assignment))

sess = tf.Session()
sess.run(tf.global_variables_initializer())

# 进行多次迭代
has_changed, cnt = True, 0
while has_changed and cnt < 300:
    cnt += 1
    has_changed, _ = sess.run([is_continue, loop])

# 查看如何分配数据
res = sess.run(point_to_centroid_assignment)
nums_in_clusters = [[] for i in range(10)]
for cluster, real_num in zip(list(res), list(y)):
    nums_in_clusters[cluster].append(real_num)

for i in range(10):
    print(Counter(nums_in_clusters[i]).most_common(3))

Extracting data/fashion/train-images-idx3-ubyte.gz
Extracting data/fashion/train-labels-idx1-ubyte.gz
Extracting data/fashion/t10k-images-idx3-ubyte.gz
Extracting data/fashion/t10k-labels-idx1-ubyte.gz
[(3, 365), (4, 180), (6, 100)]
[(4, 616), (2, 567), (6, 330)]
[(0, 539), (6, 101), (3, 9)]
[(1, 873), (3, 417), (0, 13)]
[(7, 795), (5, 220), (9, 30)]
[(6, 360), (2, 343), (0, 252)]
[(9, 437), (8, 303), (7, 129)]
[(9, 488), (5, 50), (7, 3)]
[(5, 676), (6, 96), (7, 73)]
[(8, 420), (0, 6), (4, 5)]
